This notebook contains a demonstration of how to use the Py3DOrient package for calculations of 
the Cramér-Rao lower bound (CRLB) for $\theta$ and $\phi$ angle determination in the 3d orientation determination of nanoscale objects 

In [ ]:
from CramerRaoFunctions import CRFs # import CRLB package
CRs = CRFs() # initialise an instance of said package
from NanoProp import NanoProperties # import class to simulate spectra
NP = NanoProperties() # intialise said class

### we will also use matplotlib to plot some things ###
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager
from matplotlib import cm
from scipy.stats import norm

from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.text import Annotation

In [ ]:
# first simulate the spectral response of an Au Rod of 92 nm x 40 nm
element = 'Au'
shape = 'Rod'
wavelength = np.linspace(200, 1000, 801) # every 1 nm
L = 92. # 92 nm
Width = 40. # nm
R = np.divide(L, Width) # get aspect ratio
T = 293.15 # temperature
cv = 0 # in water
AHFactor = 2.8 # ad-hoc factor to compare well with experimentally measured rod spectra
exttotal, extlongitudinal, exttransvers = NP.YuSpectra(wavelength, T, cv, L, R, element, AHFactor, shape)[:3] # get spectra

In [ ]:
# plot these spectra
plt.rcParams['figure.figsize'] = [8,6]
plt.rcParams['font.size'] = 22
plt.rcParams['axes.linewidth'] = 2 # set the value globally

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Computer Modern sans serif"],                    # use latex default serif font
    "font.sans-serif": ["Computer Modern sans serif"],  # use a specific sans-serif font
})

mpl.rcParams['text.latex.preamble'] = [
       r'\usepackage{siunitx}',   # i need upright \micro symbols, but you need...
       r'\sisetup{detect-all}',   # ...this to force siunitx to actually use your fonts
       r'\renewcommand*\familydefault{\sfdefault}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
] 

plt.plot(wavelength, exttotal, color='darkblue', label='Total Extinction')
plt.plot(wavelength, extlongitudinal, color='darkred', label='Longitudinal Plasmon')
plt.plot(wavelength, exttransvers, color='darkgreen', label='Transverse Plasmon')

plt.xlim(200,1000)
plt.xlabel('Wavelength/nm')
plt.ylabel(r'$\sigma^{ext}$/V(nm$^{-1}$)')
plt.ylim(0, 0.8)
plt.legend(loc='best', fontsize=16)
plt.grid(True,which="both",ls="--",c='gray', alpha=0.5)

plt.show(block=False)

In [ ]:
# we will have our laser centred at our longitudinal plasmon wavelength
FWHM = 2*np.sqrt(2*np.log(2)) # FWHM parameter
centrewavelength = wavelength[np.argmax(extlongitudinal)] # find the centre wavelength
laserwidth = (3*(1./FWHM)) # get our laser width
wavelengths = np.linspace(centrewavelength-laserwidth, centrewavelength+laserwidth, 1000)
IW = norm(centrewavelength, 1./FWHM).pdf(wavelengths) # laser is gaussian centred at centre wavelength
a11, a13, a33 = NP.Yua11a13a33(wavelengths, T, cv, L, R, element, AHFactor, shape, IW)

In [ ]:
# we will say we have a condenser objective of 1.3, and an imaging objective of 0.7
NACond = 1.3
NAObj = 0.7

n0 = CRs.n_m(wavelengths, T, cv)
A, B, C, H = CRs.InstrResp(NACond, NAObj, n0)[2:]
# our particle with scatter more/less dependent on the theta, this thetadepf accounts for this
thetadepf = CRs.thetadepgetscatter(a11, a13, a33, A, B, H) 

In [ ]:
# we will, for this demonstration, compare two cases - one where we have 10 photons/bin,
# and one where we have 100 photons/bin
# in both cases we will have a signal-to-background (beta) value of infinity
# we will assume we have four detectors in both cases
I1 = 10
I2 = 100
I3 = 1000
Beta = 0.
thetas = np.linspace(0, np.pi/2, 181)
phis = np.linspace(0, np.pi, 361)
time = 1
n_detectors = 4

CramerRaoMTheta10, CramerRaoMPhi10 = CRs.CramerRaoScatter(time, thetas, phis, I1, Beta,\
                        wavelengths, T, cv, NAObj, NACond, a11, a13, a33, thetadepf=thetadepf)
print('10 done')
CramerRaoMTheta100, CramerRaoMPhi100 = CRs.CramerRaoScatter(time, thetas, phis, I2, Beta,\
                        wavelengths, T, cv, NAObj, NACond, a11, a13, a33, thetadepf=thetadepf)
print('100 done')
CramerRaoMTheta1000, CramerRaoMPhi1000 = CRs.CramerRaoScatter(time, thetas, phis, I3, Beta,\
                        wavelengths, T, cv, NAObj, NACond, a11, a13, a33, thetadepf=thetadepf)
print('1000 done')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

smallfont = 10


plt.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['axes.linewidth'] = 0.5 # set the value globally

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Helvetica"],                    # use latex default serif font
    "font.sans-serif": ["Helvetica"],  # use a specific sans-serif font
})

mpl.rcParams['text.latex.preamble'] = [
       r'\usepackage{siunitx}',   # i need upright \micro symbols, but you need...
       r'\sisetup{detect-all}',   # ...this to force siunitx to actually use your fonts
       r'\renewcommand*\familydefault{\sfdefault}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
] 
s=20

def adjustlabels(ax, CLS, CS4):
    xmin,xmax,ymin,ymax = ax.axis()
    Dx = xmax-xmin
    Dy = ymax-ymin
    thresh = 0.001  # ratio in x/y range in border to discard
    # check which labels are near a border
    thresh2 = 2
    labelarr = []
    for label in CLS:
        lx, ly = label.get_position()
        labelarr.append([float(lx), float(ly)])
    labelarr = np.asarray(labelarr)

    keep_labels = []
    for label in enumerate(CLS):
        lx,ly = label[1].get_position()
        nonls = labelarr[np.arange(len(labelarr))!=label[0]]
        if xmin+thresh*Dx<lx<xmax-thresh*Dx and ymin+thresh*Dy<ly<ymax-thresh*Dy:
            if (np.any((nonls[:,0]+thresh2)<lx) or np.any(lx>(nonls[:,0]+thresh2))) and (np.any((nonls[:,1]+thresh2)<ly) or np.any(ly>(nonls[:,1]+thresh2))):
                # inlier, redraw it later
                keep_labels.append((lx,ly))

    # delete the original lines, redraw manually the labels we want to keep
    # this will leave unlabelled full contour lines instead of overlapping labels

    for cline in CS4.collections:
        cline.remove()
    for label in CLS:
        label.remove()
    return keep_labels

#===============
#  First subplot
#===============
# set up the axes for the first plot
fig, axs = plt.subplots(3, 2, sharex=True)


X, Y = np.meshgrid(np.rad2deg(thetas), np.rad2deg(phis))
Z = np.rad2deg(CramerRaoMPhi10).T
vmin=0
vmax = 180
    
levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 13), 0))

CS4 = axs[0, 0].contour(X, Y, Z, levels=levels,
                  colors=('k',),
                  linewidths=(0.5,))
CLS = axs[0, 0].clabel(CS4, fmt='%2.2f', colors='k', fontsize=smallfont)

keep_labels = adjustlabels(axs[0, 0], CLS, CS4)
    
CS4 = axs[0, 0].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[0, 0].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)

axs[0, 0].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)


axs[0, 0].set_ylabel(r'$\phi$ ($\si\degree$)')


Z = np.rad2deg(CramerRaoMPhi100).T
vmin=0
vmax = 180
    
levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 13), 0))

CS4 = axs[1, 0].contour(X, Y, Z, levels=levels,
                  colors=('k',),
                  linewidths=(0.5,))
CLS = axs[1, 0].clabel(CS4, fmt='%2.0f', colors='k', fontsize=smallfont)

keep_labels = adjustlabels(axs[1, 0], CLS, CS4)
    
CS4 = axs[1, 0].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[1, 0].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)
xfill2 = np.full(20, 90)

axs[1, 0].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[1, 0].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)


Z = np.rad2deg(CramerRaoMTheta10).T
vmin=0
vmax = 90
Z[Z>vmax]=vmax
    
levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 7), 0))

CS4 = axs[0, 1].contour(X, Y, Z, levels=levels,
                  colors=('k',),
                  linewidths=(0.5,))
CLS = axs[0, 1].clabel(CS4, fmt='%2.2f', colors='k', fontsize=smallfont)

keep_labels = adjustlabels(axs[0, 1], CLS, CS4)
    
CS4 = axs[0, 1].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[0, 1].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)
xfill2 = np.full(20, 90)

axs[0, 1].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[0, 1].scatter(xfill2, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)

axs[1, 0].set_ylabel(r'$\phi$ ($\si\degree$)')


Z = np.rad2deg(CramerRaoMTheta100).T
vmin=0
vmax = 90
Z[Z>vmax]=vmax
    
levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 7), 0))

CS4 = axs[1, 1].contour(X, Y, Z, levels=levels,
                  colors=('k',),
                  linewidths=(0.5,))
CLS = axs[1, 1].clabel(CS4, fmt='%2.0f', colors='k', fontsize=smallfont)

keep_labels = adjustlabels(axs[1, 1], CLS, CS4)

CS4 = axs[1, 1].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[1, 1].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)
xfill2 = np.full(20, 90)

axs[1, 1].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[1, 1].scatter(xfill2, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)

Z = np.rad2deg(CramerRaoMPhi1000).T
vmin=0
vmax = 90
Z[Z>vmax]=vmax
    
levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 13), 0))

CS4 = axs[2, 0].contour(X, Y, Z, levels=levels,
                  colors=('white',),
                  linewidths=(0.5,))
CLS = axs[2, 0].clabel(CS4, fmt='%2.2f', colors='w', fontsize=smallfont)

keep_labels = adjustlabels(axs[2, 0], CLS, CS4)
    
CS4 = axs[2, 0].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[2, 0].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)

axs[2, 0].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[2, 0].set_ylabel(r'$\phi$ ($\si\degree$)')
axs[2, 0].set_xlabel(r'$\theta$ ($\si\degree$)')


Z = np.rad2deg(CramerRaoMTheta1000).T
vmin=0
vmax = 90
Z[Z>vmax]=vmax

levels = np.unique(np.around(np.geomspace((np.nanmin(Z)), (vmax), 7), 0))

CS4 = axs[2, 1].contour(X, Y, Z, levels=levels,
                  colors=('k',),
                  linewidths=(0.5,))
CLS = axs[2, 1].clabel(CS4, fmt='%2.2f', colors='k', fontsize=smallfont)

keep_labels = adjustlabels(axs[2, 1], CLS, CS4)
    
CS4 = axs[2, 1].contour(X, Y, Z, levels=levels,
                  colors='k',
                  linewidths=(0.5,))
CLS = axs[2, 1].clabel(CS4, fmt='%2.0f', colors='k',  fontsize=smallfont, manual=keep_labels)
xfill = np.zeros(20)
yfill = np.linspace(0, 180, 20)
xfill2 = np.full(20, 90)

axs[2, 1].scatter(xfill, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[2, 1].scatter(xfill2, yfill, marker='o', lw=0.5, facecolors='none', edgecolors='k', s=s)
axs[2, 1].set_xlabel(r'$\theta$ ($\si\degree$)')

axs[0, 0].annotate(r'$\sigma_{\phi\phi}$, n$_{\mathrm{tot}}=10$', (70, 160))
axs[0, 1].annotate(r'$\sigma_{\theta\theta}$, n$_{\mathrm{tot}}=10$', (70, 160))

axs[1, 0].annotate(r'$\sigma_{\phi\phi}$, n$_{\mathrm{tot}}=100$', (70, 160))
axs[1, 1].annotate(r'$\sigma_{\theta\theta}$, n$_{\mathrm{tot}}=100$', (70, 160))

axs[2, 0].annotate(r'$\sigma_{\phi\phi}$, n$_{\mathrm{tot}}=1000$', (70, 160))
axs[2, 1].annotate(r'$\sigma_{\theta\theta}$, n$_{\mathrm{tot}}=1000$', (70, 160))

for i in np.arange(3):
    for j in np.arange(2):
        axs[i,j].set_yticks([0, 45, 90, 135, 180])
        axs[i,1].set_yticklabels([])
        

plt.tight_layout()
plt.show(block=False)